In [37]:
# Initial imports
import pandas as pd
import hvplot.pandas
from pathlib import Path
from collections import Counter
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


In [38]:
import warnings
warnings.filterwarnings('ignore')

In [39]:
#Load Dataframe
data = Path('Loan_Predictions/Loans_Data_ML.csv')
df = pd.read_csv(data)
df.head()

,Loan_Amt,Loan_Term,Monthly_Payment,Credit_Grade,Employement_Length,Annual_Income,DTI_Ratio,Application_Type,Joint_Annual_Income,Joint_DTI_Ratio,...,Delinquent_Amount,#_Mortgage_Accounts,#_Bankruptcies,#_Tax_Liens,Balance_Exc_Mortgage,Joint_Revolving_Balance,Open_Acc_IL24m,Open_RevAcc_24m,Acc_Curr_PD30days,Loan_Status
0,3600,36,111.97,A4,10+ years,120000.0,18.90,INDIVIDUAL,0.0,0.0,...,0,0,1,0,36506,0.0,14,11,0.0,Issued
1,15000,60,356.78,C4,10+ years,125000.0,17.25,INDIVIDUAL,0.0,0.0,...,0,2,0,0,69364,0.0,3,3,0.0,Issued
2,8400,36,276.56,B3,8 years,50000.0,15.63,INDIVIDUAL,0.0,0.0,...,0,4,0,0,51591,0.0,5,12,0.0,Issued
3,4000,36,130.00,B2,2 years,50000.0,33.61,INDIVIDUAL,0.0,0.0,...,0,6,0,0,136208,0.0,7,4,0.0,Issued
4,6000,36,185.93,A3,3 years,125000.0,9.25,INDIVIDUAL,0.0,0.0,...,0,0,0,0,60622,0.0,15,3,0.0,Current


In [40]:
# Remove the `Issued` loan status
issued_mask = df['Loan_Status'] != 'Issued'
df = df.loc[issued_mask]
issued_mask = df['Loan_Status'] != 'Fully Paid'
df = df.loc[issued_mask]

# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,Loan_Amt,Loan_Term,Monthly_Payment,Credit_Grade,Employement_Length,Annual_Income,DTI_Ratio,Application_Type,Joint_Annual_Income,Joint_DTI_Ratio,...,Delinquent_Amount,#_Mortgage_Accounts,#_Bankruptcies,#_Tax_Liens,Balance_Exc_Mortgage,Joint_Revolving_Balance,Open_Acc_IL24m,Open_RevAcc_24m,Acc_Curr_PD30days,Loan_Status
0,6000,36,185.93,A3,3 years,125000.0,9.25,INDIVIDUAL,0.0,0.0,...,0,0,0,0,60622,0.0,15,3,0.0,low_risk
1,12000,36,370.48,A2,10+ years,56000.0,17.90,INDIVIDUAL,0.0,0.0,...,0,5,0,0,21540,0.0,5,0,0.0,low_risk
2,10500,60,345.15,G2,< 1 year,45000.0,17.25,INDIVIDUAL,0.0,0.0,...,0,0,1,0,22148,0.0,7,13,0.0,low_risk
3,18000,36,542.07,A1,< 1 year,155000.0,9.08,INDIVIDUAL,0.0,0.0,...,0,0,0,0,61257,0.0,4,1,0.0,low_risk
4,1600,36,53.72,C1,1 year,60000.0,12.62,INDIVIDUAL,0.0,0.0,...,0,0,0,0,27231,0.0,6,10,0.0,low_risk


## Split the Data into Training and Testing

In [41]:
# Create our features

X = df.drop(columns='Loan_Status', axis=1)
X = pd.get_dummies(X)

# Create our target
y = df["Loan_Status"]

In [42]:
X.describe()

,Loan_Amt,Loan_Term,Monthly_Payment,Annual_Income,DTI_Ratio,Joint_Annual_Income,Joint_DTI_Ratio,#_Open_Accounts,#_Delinquent_Accounts,Total_Amount_in_Collection,...,Employement_Length_5 years,Employement_Length_6 years,Employement_Length_7 years,Employement_Length_8 years,Employement_Length_9 years,Employement_Length_< 1 year,Employement_Length_None,Application_Type_DIRECT_PAY,Application_Type_INDIVIDUAL,Application_Type_JOINT
count,79457.000000,79457.000000,79457.000000,7.945700e+04,79457.000000,7.945700e+04,79457.000000,79457.000000,79457.000000,79457.000000,...,79457.000000,79457.000000,79457.000000,79457.000000,79457.000000,79457.000000,79457.000000,79457.000000,79457.000000,79457.000000
mean,14904.629233,42.052783,452.540493,8.276659e+04,21.767357,5.093544e+03,0.777891,11.799049,0.006507,252.732887,...,0.062499,0.044062,0.033389,0.033125,0.036888,0.071372,0.070290,0.000277,0.956165,0.043558
std,9369.742250,10.422667,283.002287,2.308469e+05,174.594217,2.668625e+04,3.912830,5.734920,0.085558,3788.664649,...,0.242062,0.205233,0.179652,0.178964,0.188488,0.257447,0.255636,0.016637,0.204729,0.204111
min,1000.000000,36.000000,30.120000,0.000000e+00,-1.000000,0.000000e+00,0.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8000.000000,36.000000,247.110000,4.900000e+04,12.330000,0.000000e+00,0.000000,8.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
50%,12000.000000,36.000000,373.560000,7.000000e+04,18.070000,0.000000e+00,0.000000,11.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
75%,20000.000000,60.000000,604.240000,1.000000e+05,24.410000,0.000000e+00,0.000000,15.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
max,40000.000000,60.000000,1715.420000,6.100000e+07,9999.000000,1.210000e+06,61.900000,71.000000,3.000000,932461.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [43]:
# Check the balance of our target values
y.value_counts()

low_risk     78897
high_risk      560
Name: Loan_Status, dtype: int64

In [44]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)
X_train.shape

(59592, 72)

In [45]:
pip install imblearn


Note: you may need to restart the kernel to use updated packages.


### Oversampling

## Naive Random Oversampling

In [46]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
Counter(y_resampled)


Counter({'low_risk': 59172, 'high_risk': 59172})

In [47]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [48]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.5601792504073873

In [49]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[  102,    38],
       [11997,  7728]])

In [50]:
# Print the imbalanced classification report
# Naive Random Oversampling
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.73      0.39      0.02      0.53      0.30       140
   low_risk       1.00      0.39      0.73      0.56      0.53      0.28     19725

avg / total       0.99      0.39      0.73      0.56      0.53      0.28     19865



## SMOTE Oversampling

In [51]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy='auto').fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({'low_risk': 59172, 'high_risk': 59172})

In [52]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [53]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.5647130182871627

In [54]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[   94,    46],
       [10691,  9034]])

In [55]:
# Print the imbalanced classification report
# SMOTE Oversampling
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.67      0.46      0.02      0.55      0.31       140
   low_risk       0.99      0.46      0.67      0.63      0.55      0.30     19725

avg / total       0.99      0.46      0.67      0.62      0.55      0.30     19865



## Undersampling

In [56]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=78)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=78)

In [57]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

0.5647130182871627

In [58]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[   94,    46],
       [10691,  9034]])

In [59]:
# Print the imbalanced classification report
# Undersampling
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.67      0.46      0.02      0.55      0.31       140
   low_risk       0.99      0.46      0.67      0.63      0.55      0.30     19725

avg / total       0.99      0.46      0.67      0.62      0.55      0.30     19865



## Combination (Over and Under) Sampling


In [60]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [61]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

0.5647130182871627

In [62]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[   94,    46],
       [10691,  9034]])

In [63]:
# Print the imbalanced classification report
# Combination (Over and Under) Sampling
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.67      0.46      0.02      0.55      0.31       140
   low_risk       0.99      0.46      0.67      0.63      0.55      0.30     19725

avg / total       0.99      0.46      0.67      0.62      0.55      0.30     19865

